In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Parameters (num_frames / fps = duration of video in seconds)
width, height = 400, 336
num_frames = 30 
text = "243"
font_path = "arial.ttf"
output_video = "BotCheck.avi"
shift_speed = 3 
fps= 30
font_size = 200

# Function to wrap text (relevant for longer text)
def wrap_text(text, font, max_width):
    lines = []
    words = text.split()
    while words:
        line = ''
        while words and font.getbbox(line + words[0])[2] <= max_width:
            line += (words.pop(0) + ' ')
        lines.append(line)
    return lines

# Generates a random black and white noise background
base_noise = np.random.randint(0, 2, (int(height), int(width)), dtype=np.uint8) * 255


mask_img = Image.new("L", (int(width), int(height)), 0)
draw_mask = ImageDraw.Draw(mask_img)

font = ImageFont.truetype(font_path, font_size)


wrapped_text = wrap_text(text, font, int(width))

# Draw wrapped text
y_offset = 10
for line in wrapped_text:
    bbox = draw_mask.textbbox((0, 0), line, font=font)
    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]
    text_x = (int(width) - text_w) + 20
    text_y = y_offset + (int(height) - text_h * len(wrapped_text)) / 2 -40
    draw_mask.text((text_x, text_y), line, font=font, fill=255)
    y_offset += text_h


text_mask = np.array(mask_img)

# Note: Other codecs generate smaller files, but flickering may occur
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Generate frames
for i in range(num_frames):
    shift_x = i * shift_speed
    # Shift background
    shifted_bg = np.roll(base_noise, shift_x, axis=1)
    # Place unshifted base noise in the text region
    frame_gray = np.where(
        text_mask[..., None] == 255,
        base_noise[..., None],
        shifted_bg[..., None]
    )
    frame_bgr = cv2.cvtColor(frame_gray.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    video.write(frame_bgr)

video.release()
print(f"Video saved as {output_video}")

Video saved as V2BotCheckNumberOnly.avi


In [8]:
import imageio

# Basic procedure to convert video to GIF
def convert_video_to_gif(video_path, gif_path, duration_seconds=1, target_fps=30):
    video_reader = cv2.VideoCapture(video_path)
    frames = []
    
    # Calculate frames to capture
    start_time = 0
    end_time = start_time + duration_seconds
    frame_count = 0
    
    while True:
        success, frame = video_reader.read()
        if not success:
            break
            
        current_time = frame_count / video_reader.get(cv2.CAP_PROP_FPS)
        if current_time > end_time:
            break
            
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_count += 1
    
    video_reader.release()
    
    # Save as GIF with explicit loop setting
    imageio.mimsave(
        gif_path, 
        frames, 
        fps=target_fps,
        loop=0,  # 0 means infinite loop
        duration=1/target_fps  # explicit frame duration
    )

# Convert first 3 seconds to smooth GIF
output_gif = 'output.gif'
convert_video_to_gif(output_video, output_gif)